<a href="https://colab.research.google.com/github/AnIsAsPe/LDATopicModeling_pyspark/blob/main/LDA_con_sklearn_noticias_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalar y cargar bibliotecas

In [1]:
!pip install pyLDAvis  #biblioteca que extrae información de un modelo LDA para obtener una visualización interactiva

     |████████████████████████████████| 1.7MB 5.3MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 9.9MB 16.9MB/s 
     |████████████████████████████████| 15.3MB 191kB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-cp37-none-any.whl size=136897 sha256=1ba4da51f30d53d51f40f9286db435fde60c5b936410269836adc6234d567a96
  Stored in directory: /root/.cache/pip/wheels/a0/9c/fc/c6e00689d35c82cf96a8adc70edfe7ba7904374fdac3240ac2
Successfully built pyLDAvis
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.20.3 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0

In [1]:
import pandas as pd
import numpy as np

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer

import pyLDAvis
import matplotlib.pyplot as plt 
import seaborn as sns  

nltk.download('stopwords')
nltk.download('wordnet')  #WordNetLemmatizer
nltk.download('punkt')


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_lda.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).ep

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Lectura y exploración de datos

En colab, al instalar la librería pyLDAvis causa un conflicto con Pandas. Si esto se refleja al correr la siguiente linea es necesario reiniciar el entorno de ejecución.

In [2]:
df = pd.read_csv('/content/drive/MyDrive/Datos/abcnews-date-text.csv', parse_dates=['publish_date'] )
print(df.shape)
df.head()

(1226258, 2)


,publish_date,headline_text
0,2003-02-19,aba decides against community broadcasting lic...
1,2003-02-19,act fire witnesses must be aware of defamation
2,2003-02-19,a g calls for infrastructure protection summit
3,2003-02-19,air nz staff in aust strike for pay rise
4,2003-02-19,air nz strike to affect australian travellers


In [3]:
df.publish_date.min(), df.publish_date.max()

(Timestamp('2003-02-19 00:00:00'), Timestamp('2020-12-31 00:00:00'))

In [4]:
# promedio de noticias por día
len(df)/(df.publish_date.max()-df.publish_date.min()).days


187.93226053639847

In [10]:

filtro = (df['publish_date']>= '2020-01-01')&(df['publish_date']<= '2020-12-31')
df = df[filtro]
len(df)

40240

# Vectorización de textos

In [11]:
def preprocesar(texto):
  #convierte a minúsculas
  texto = (texto).lower()

  #elimina stopwords
  stop = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
  texto = stop.sub('', texto) 

  #lematizar y quedarnos con palabras que tengan más de tres caracteres
  lemmatizer = WordNetLemmatizer()
  texto = texto.split()
  texto = ' '.join([lemmatizer.lemmatize(i) for i in texto if len(i)>3])
  
  return(texto)

In [12]:
%%time
df['headline_pp']=df['headline_text'].apply(preprocesar)
df.head()

CPU times: user 8.86 s, sys: 738 ms, total: 9.6 s
Wall time: 9.63 s


In [13]:
vectorizer = TfidfVectorizer(min_df=3, ngram_range=(1,2))
BOW = vectorizer.fit_transform(df['headline_pp'])
BOW.shape

(40240, 16579)

# Entrenamiento del modelo

In [19]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=5,learning_method='online',random_state=42,max_iter=50) 

In [20]:
%%time
lda_top=lda_model.fit_transform(BOW) # entrena el modelo y obtienela matriz documento-topico

CPU times: user 4min 1s, sys: 1.62 s, total: 4min 2s
Wall time: 4min 2s


### Matriz documento tópico

In [27]:
doc_top = pd.DataFrame(lda_top)
print(doc_top.shape)
doc_top.head()

(40240, 5)


,0,1,2,3,4
0,0.394430,0.380654,0.074977,0.074743,0.075196
1,0.199485,0.630972,0.056487,0.056418,0.056639
2,0.165149,0.666773,0.056001,0.055938,0.056139
3,0.062683,0.458073,0.062562,0.062565,0.354117
4,0.502484,0.067217,0.067445,0.295285,0.067569


# Matriz tópicos-tokens

In [26]:
lda_model.components_

(5, 16579)

In [36]:
top_tokens= pd.DataFrame(lda_model.components_,
                         columns=vocabulario)
print(top_tokens.shape)
top_tokens.head()

(5, 16579)


,0501,1000,10000,100000,100k,100m,100th,100th birthday,1080,10th,10yo,11th,12th,130,130 billion,13th,14th,14yo,14yo girl,150,150 million,1500,15th,15yo,16th,17th,17yo,18th,18yo,1970s,1980s,1983,1988,1990s,1995,1996,1999,19th,19yo,2000,...,zahra,zealand,zealand auckland,zealand australia,zealand coronavirus,zealand covid,zealand election,zealand host,zealand jacinda,zealand lift,zealand record,zealand travel,zealand vote,zealand white,zealand woman,zealander,zempilas,zero,zero carbon,zero case,zero coronavirus,zero covid,zero death,zero emission,zero local,ziffer,ziggy,ziggy ramo,zillmere,zillmere brawl,zimbabwe,zlate,zlate cvetanovski,zodiac,zombie,zone,zoo,zoom,zuckerberg,zverev
0,0.200033,9.111763,4.077624,0.202004,0.207123,1.762069,0.200896,0.200714,0.200452,0.200016,0.217593,0.200318,0.200063,0.201344,0.201344,0.200033,0.200057,0.203625,0.206042,1.722284,1.432123,0.208091,0.200016,0.201256,0.200024,0.201364,0.203269,0.200017,0.202239,0.212366,1.510152,1.181604,2.978502,0.200035,1.805999,0.200243,0.205638,0.208218,1.599370,0.202326,...,1.579991,0.204959,0.200536,2.387356,0.200334,0.200030,0.200187,0.202449,0.200492,0.200029,0.200022,0.201550,0.201329,0.200260,0.201081,0.201300,1.549461,0.201580,0.202691,0.200725,0.200107,0.200038,0.205104,0.200922,0.200029,0.200222,0.200970,0.200970,4.427625,0.203395,0.206559,0.200362,0.200362,0.200036,0.202082,0.203077,0.216620,0.216158,0.200970,0.201453
1,2.049034,0.206527,0.216758,4.210217,0.202938,0.202270,3.000400,2.540759,0.200025,0.200013,2.217388,0.202924,2.169513,1.429844,1.429844,0.204395,0.200359,2.700029,1.922267,0.202148,0.203226,0.202317,0.200013,0.201844,0.202160,0.208982,0.201060,0.200014,0.200504,0.200646,0.201935,0.201900,0.201337,0.203737,0.206347,0.200683,0.214095,0.200889,0.200042,9.559791,...,0.200019,0.212501,2.149336,0.200410,0.204782,1.942133,0.200180,0.200426,0.200604,1.680792,3.312040,0.205136,0.200024,1.842353,0.200582,0.207586,0.200028,0.209004,0.200018,0.204317,3.254015,0.208064,1.522901,0.200249,0.204849,0.200027,0.200988,0.200988,0.200555,0.200022,2.040365,1.360361,1.360361,0.200034,0.216106,0.210442,0.206090,6.471129,0.200728,0.200686
2,0.200030,0.203799,0.200577,0.203309,0.200888,0.202516,0.203572,0.203142,0.201533,2.913399,0.202492,3.147643,0.216703,0.200411,0.200411,3.188384,0.215572,0.200462,0.200270,0.201141,0.200917,0.203577,3.469768,0.200958,4.150737,3.141834,0.200037,2.415706,0.201472,2.106921,0.200061,0.201017,0.202058,0.202793,0.200042,0.200300,0.200068,3.427343,0.200444,0.203867,...,0.200020,0.201999,0.200023,0.202896,0.201749,0.200692,0.200637,0.201056,0.200103,0.200325,0.200020,0.201078,0.200830,0.200025,0.201052,0.200219,0.200453,32.090546,1.333141,10.659872,0.202130,1.924221,0.211495,3.014000,2.630693,3.121896,0.200028,0.200028,0.200739,0.200025,0.202808,0.200418,0.200418,2.219681,0.200056,14.002050,2.254762,0.203377,1.801844,0.201464
3,0.200034,0.202232,0.222078,0.203705,0.204407,0.200378,0.202035,0.202438,0.200029,0.200017,0.204440,0.200678,0.201163,0.200033,0.200033,0.200034,0.200059,0.205255,0.203716,0.201771,0.201841,2.975499,0.200017,2.178724,0.200025,0.200045,1.552296,0.200018,1.731663,0.206935,0.257660,0.200851,0.204406,2.396294,0.201601,0.200518,0.200316,0.202295,0.212620,0.202369,...,0.201256,0.202030,0.200025,0.205940,0.200269,0.201621,0.200018,0.202424,0.200086,0.200441,0.200023,0.200035,0.200030,0.201213,0.200666,0.200371,0.204481,0.200863,0.200603,0.200142,0.200023,0.200038,0.200054,0.200205,0.200029,0.203935,1.613704,1.613704,0.209704,1.665125,0.202735,0.204505,0.204505,0.201003,2.193464,0.201625,0.201290,0.202303,0.200026,0.205082
4,0.200031,0.201808,0.207142,0.209033,2.418162,0.200274,0.200279,0.200328,1.875860,0.200015,0.203521,0.201588,0.200787,0.202527,0.202527,0.202351,1.806130,0.202391,0.200041,0.200357,0.200513,0.213428,0.200015,0.208555,0.200409,0.201614,0.205406,0.200016,0.200028,0.204303,0.200063,1.151549,0.200126,0.200951,0.200048,1.393689,1.588613,0.200713,0.201878,0.204822,...,0.201959,61.969349,0.202153,0.209476,5

## Perplejidad del modelo

In [21]:

%%time
lda_model.perplexity(BOW, sub_sampling = False)

CPU times: user 3.93 s, sys: 52.3 ms, total: 3.99 s
Wall time: 3.97 s


18307.9716880483

# Visualización del modelo

In [22]:
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

pyLDAvis.sklearn.prepare(lda_model, BOW, vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.208227 -0.093653       1        1  22.579047
2     -0.014750  0.075250       2        1  20.536598
4      0.146513 -0.004906       3        1  19.740019
0     -0.224926 -0.218564       4        1  18.673722
3     -0.115064  0.241874       5        1  18.470613, topic_info=             Term        Freq       Total Category  logprob  loglift
11760  queensland  267.000000  267.000000  Default  30.0000  30.0000
2502         case  257.000000  257.000000  Default  29.0000  29.0000
15219       trump  216.000000  216.000000  Default  28.0000  28.0000
4070        covid  453.000000  453.000000  Default  27.0000  27.0000
5369     election  165.000000  165.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
4237        crash   56.722948   91.097753   Topic5  -5.8127   1.2152
7408         home   59.369696  131.339664   Topic5  -5.7671   0.8950
16291       woman   60.360555  154.539935   Topic5  -5.7506   0.7489
5830       family   53.338171  108.926604   Topic5  -5.8742   0.9750
4542        death   50.523167  165.528832   Topic5  -5.9285   0.5023

[244 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
145        5  0.987921      abuse
186        5  0.989099    accused
358        4  0.992740       aged
359        4  0.995184  aged care
405        5  0.989490       alan
...      ...       ...        ...
16291      5  0.388249      woman
16354      1  0.203816     worker
16354      3  0.686538     worker
16354      5  0.107272     worker
16540      3  0.994264    zealand

[290 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 5, 1, 4])

## Matriz - Documento Término